# Visualize models performance 

In [ ]:
import numpy as np
import pandas as pd
import anndata as adata

from tqdm import tqdm
from pathlib import Path
from typing import Tuple, List, Dict

import plotly.graph_objects as go
import plotly as plotly
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity as skl_cosine

from scipy.stats import pearsonr
from scipy.spatial.distance import cosine as scipy_cosine
from scipy.spatial.distance import braycurtis, cdist
from math import sqrt

%load_ext blackcellmagic

In [ ]:
# Prefix to visualizations folder
viz_prefix = "???/deconvolution_benchmarking/visualizations"

# Prefix to the experiment we're plotting
prefix = "???/deconvolution_benchmarking/01_purity_levels_experiment/include_normal_epithelial"

# Tumour purity levels
purity_levels = np.arange(0.05, 1, 0.05).round(3).tolist()

# Major cell types
c_types = [
    "Cancer Epithelial",
    "Normal Epithelial",
    "T-cells",
    "B-cells",
    "Myeloid",
    "CAFs",
    "Endothelial",
    "PVL",
    "Plasmablasts",
]

# Methods order are universal across figures
methods_order = [
    "BayesPrism",
    "Scaden",
    "MuSiC",
    "hspe",
    "DWLS",
    "CBX",
    "Bisque",
    "EPIC",
    "CPM",
]

### Load groundtruth

In [ ]:
# Load truth.csv
truth_df = pd.read_csv(
    Path(prefix).joinpath("data/results/truth.csv"), sep="\t", index_col=0
)
truth_df = truth_df[c_types]

# Pivot longer for when we need it
truth_copy_df = truth_df.copy().sample(frac=0.05, random_state=41)
truth_copy_df["purity_level"] = truth_copy_df["Cancer Epithelial"]

pivot_truth_df = (
    truth_copy_df.reset_index()
    .melt(id_vars=["index", "purity_level"], value_vars=c_types)
    .rename(columns={"index": "mixture_id", "variable": "cell_type", "value": "truth"})
    .set_index(["mixture_id", "cell_type"])
)

### Extract colour pallete

In [ ]:
# Extract colour pallete
ctype_colour_pallete_df = pd.read_csv(
    Path(prefix).joinpath("data/Whole_miniatlas_colour_pallete.csv"), sep="\t"
)

# Convert to dictionary
ctype_colour_pallete_d = {
    row["all_celltype"]: {"fill": row["fill"], "line": row["line"]}
    for i, row in ctype_colour_pallete_df.iterrows()
}

## [Supp Fig 4b]. Pearson's r over tumour purity levels

### Collect results and calculate metrics
All (predictions-groundtruth) DataFrames are stored in the same format [Mixtures x Cell_types] for all methods <br>
Specific which methods we'd like to collect results for<br>
The collect results and generate performance metrics nicely

In [ ]:
methods = [
    "hspe",
    "music",
    "cpm",
    "cbx",
    "scaden",
    "epic",
    "bisque",
    "dwls",
    "bprism_v2",
]

pearsonr_l = []

In [ ]:
for method in tqdm(methods):
    res_df = pd.read_csv(
        Path(prefix).joinpath(f"data/results/{method}.csv"), sep="\t", index_col=0
    )
    res_df = res_df[c_types]
    res_df[res_df < 0] = 0

    print(f"Generating performance metrics for {method}")

    # Iterate over purity levels and calculate performance metrices for each level
    pearrson_d = {}

    for pur_lvl in tqdm(purity_levels):
        subset_truth_df = truth_df[
            truth_df["Cancer Epithelial"] == pur_lvl
        ].sort_index()
        subset_res_df = res_df[res_df.index.isin(subset_truth_df.index)].sort_index()

        # Calculate Pearson's r
        pearson_r = pearsonr(
            subset_truth_df.values.flatten(), subset_res_df.values.flatten()
        )
        pearrson_d[pur_lvl] = pearson_r[0]

    pearson_series = pd.Series(pearrson_d)
    pearson_series.name = method
    pearsonr_l.append(pearson_series)

In [ ]:
# Concatenate all metrics across purity levels into total metrics DataFrames
pearsonr_df = pd.concat(pearsonr_l, axis=1).T

#### Pivot the metrics that we'd like to plot

In [ ]:
# Pearson's r
pivot_pearsonr_df = pd.melt(
    pearsonr_df.reset_index(), id_vars=["index"], value_vars=purity_levels
)
pivot_pearsonr_df.columns = ["Method", "Purity Level", "Pearson's r"]

In [ ]:
# Merger all metrics into one beautiful metrics DataFrame
metrics_df = pivot_pearsonr_df.copy()
metrics_df["Pearsonr_round"] = metrics_df["Pearson's r"].round(2)

# Replace tools by their correct names
metrics_df.replace(
    {
        "scaden": "Scaden",
        "music": "MuSiC",
        "cbx": "CBX",
        "bisque": "Bisque",
        "dwls": "DWLS",
        "epic": "EPIC",
        "cpm": "CPM",
        "bprism_v2": "BayesPrism",
    },
    inplace=True,
)

# Convert purity level to % scale and categorical
metrics_df["Purity Level"] = (metrics_df["Purity Level"] * 100).astype(int)
metrics_df["Purity Level"] = metrics_df["Purity Level"].astype("category")
metrics_df["Purity Level"].cat.reorder_categories(
    [int(i * 100) for i in purity_levels], inplace=True
)

### Plot heatmap of Pearson's r over tumour purity levels

In [ ]:
# Save source data
metrics_df[["Method", "Purity Level", "Pearson's r"]].to_csv(
    Path(viz_prefix).joinpath("source_data/supp_figure_4a.tsv"), sep="\t"
)

In [ ]:
pivot_pearson_df = pd.pivot_table(
    metrics_df[["Method", "Purity Level", "Pearson's r"]],
    values="Pearson's r",
    index=["Method"],
    columns=["Purity Level"],
)

In [ ]:
fig = ff.create_annotated_heatmap(
    z=pivot_pearson_df.values,
    annotation_text=pivot_pearson_df.values.round(2),  # Annotate with Pearson's r
    zmin=0,
    zmax=1,
    x=pivot_pearson_df.columns.tolist(),  # Rows are methods
    y=pivot_pearson_df.index.tolist(),  # Columns are cell types
    colorscale="blues_r",
    showscale=True,
    hoverinfo="text",
    text=pivot_pearson_df.round(2).values,
    colorbar=dict(
        title="Pearson's r",
        titlefont_size=14,
        ticks="outside",
        ticksuffix="",
        tickfont_size=12,
        dtick=0.1,
        orientation="v",
    ),
)

fig["layout"].update(
    margin=dict(t=0, l=0, r=0, b=0),
    plot_bgcolor="rgba(0,0,0,0)",
    font=dict(size=12, color="black"),
    xaxis=dict(
        title="Tumour purity levels",
        titlefont_size=14,
        title_standoff=5,
        ticks="outside",
        showticklabels=True,
        tickmode="array",
        tickwidth=0.75,
        ticklen=3,
        tickvals=[int(i * 100) for i in purity_levels],
        tickfont_size=12,
        linecolor="black",
        linewidth=0.75,
        side="bottom",
    ),
    yaxis=dict(
        title="Methods",
        titlefont_size=14,
        title_standoff=1,
        linecolor="black",
        linewidth=0.75,
        ticks="outside",
        tickwidth=0.75,
        ticklen=3,
        tickfont_size=12,
        categoryorder="array",
        categoryarray=methods_order[::-1],
    ),
)

# Save into PNG and SVG
fig.write_image(
    Path("figures/supp_figures/supp_fig_4b").with_suffix(".svg"),
    width=900,
    height=300,
    scale=5,
)

## [Supp Fig 4c]. RMSE over tumour purity level across cell type

In [ ]:
# We don't need to show all 19 tumour purities for each tool. Just half of it will be fine
# Getting tumour purity levels with intervals of 15% instead of 10%
reduced_purity_levels = np.arange(0.05, 1, 0.15).round(3).tolist()

methods = [
    "bprism_v2",
    "music",
    "cpm",
    "cbx",
    "scaden",
    "epic",
    "bisque",
    "dwls",
    "hspe",
]

ctypes_order = [
    "Plasmablasts",
    "PVL",
    "CAFs",
    "Endothelial",
    "Myeloid",
    "B-cells",
    "T-cells",
    "Normal Epithelial",
    "Cancer Epithelial",
]

In [ ]:
def collapse_epithelial(source_df: pd.DataFrame) -> pd.DataFrame:
    """Collapse Normal Epithelial and Cancer Epithelial into Epithelial

    Args:
        source_df:      predictions or groundtruth DataFrame. Rows as mixtures, columns as cell types
    """
    # Copy provided DataFrame so we don't alter it
    df = source_df.copy()

    # Collapse Cancer and Normal Epithelial, and then drop these cols
    df["Epithelial"] = df["Normal Epithelial"] + df["Cancer Epithelial"]
    df.drop(["Normal Epithelial", "Cancer Epithelial"], axis=1, inplace=True)

    return df

In [ ]:
def calculate_metrics(
    subset_truth_df: pd.DataFrame, subset_res_df: pd.DataFrame, c_types: List
) -> pd.DataFrame:
    """Iterate over provided cell types and calculate peformance metrics of predictions against groundtruth
    The method assumes that provided cell types are consistent across both prediction and groundtruth DataFrame

    Args:
        - subset_truth_df:     groundtruth DataFrame, purity-level-specific
        - subset_res_df:     predictions DataFrame, purity-level-specific
        - c_types:             cell types to iterate over
    """
    # Create an empty list to hold peformance metrics of each cell type
    metrics_series_l = []

    # Iterate over cell types and calcuate RMSE + MAE + Cosine
    for c_type in c_types:
        # Re-arrange colums both predictions and groundtruth in the same order
        ctype_truth_df = subset_truth_df[c_type]
        ctype_preds_df = subset_res_df[c_type]

        # RMSE
        rmse = sqrt(mean_squared_error(ctype_truth_df * 100, ctype_preds_df * 100))

        # MAE
        mae = abs(ctype_truth_df - ctype_preds_df).median() * 100

        # RPE
        rpe = (
            abs(ctype_truth_df - ctype_preds_df) / ctype_truth_df.replace({0: 0.0001})
        ).median()

        metrics_series_l.append(pd.Series([rmse, mae, rpe], name=c_type))

    # Concatenate metrics across cell types
    method_metrics_df = pd.concat(metrics_series_l, axis=1)
    method_metrics_df.index = ["RMSE", "MAE", "RPE"]

    return method_metrics_df

In [ ]:
def plot_rmse_heatmap(
    avg_diff_df: pd.DataFrame,
    outfile_name: str,
    metric: str,
    metric_suffix: str,
    colorscale: str,
    c_types: List,
    plot_w: int,
    plot_h: int,
    z_range: List = [0, 50],
    dticks: int = 10,
    auto_open: bool = True,
) -> None:
    """Plot heatmap of Mean Absolute Error across tumour purity levels

    Args:
        - avg_diff_df:        DataFrame holding MAE over tumour purity levels
        - outfile_name:       name of output html and png files
        - c_types:            cell types in a specific order we'd like to appear on the y-axis
        - z_range:            Maximum error (between Scaden, CBX and EPIC) is ~56%
                              so we only need to set maximum zaxis to 50%.
                              This ensure extreme errors are very red on the scale
        - auto_open:          Whether to open html after creation or not

    """
    # Create annotated heatmap object with plotly
    fig = ff.create_annotated_heatmap(
        z=avg_diff_df.values,
        # Annotate each cell in the heatmap with the corresponding labels
        annotation_text=avg_diff_df.values.round(1).astype(str),
        zmin=z_range[0],
        zmax=z_range[1],
        x=(avg_diff_df.columns * 100).astype(int).tolist(),  # Rows are purity levels
        y=avg_diff_df.index.tolist(),  # Columns are cell types
        colorscale=colorscale,
        showscale=False,
        hoverinfo="text",
        text=avg_diff_df.values.round(1),
        colorbar=dict(
            title=metric,
            ticks="outside",
            ticksuffix=metric_suffix,
            dtick=dticks,
            orientation="h",
            ticklen=2,
        ),
    )

    # Update axes
    fig.update_xaxes(
        title="Tumour purity levels (%)",
        title_font_size=8,
        title_standoff=5,
        ticks="outside",
        tickfont_size=7,
        showticklabels=True,
        ticklen=2,
        tickwidth=0.5,
        tickmode="array",
        tickvals=(avg_diff_df.columns * 100).astype(int).tolist(),
        linecolor="black",
        linewidth=0.5,
        side="bottom",
    )
    fig.update_yaxes(
        title="Cell Types",
        title_font_size=8,
        title_standoff=5,
        linecolor="black",
        linewidth=0.5,
        categoryorder="array",
        categoryarray=c_types,  # Order cell types by linages
        ticks="outside",
        showticklabels=True,
        ticklen=2,
        tickwidth=0.5,
        tickfont_size=7,
    )

    # Update layout
    fig["layout"].update(
        margin=dict(t=0, l=0, r=0, b=0),
        font_size=6,
        plot_bgcolor="rgba(0,0,0,0)",
        font_color="black",
    )

    # Save offline mode
    fig.write_image(
        Path(f"{outfile_name}").with_suffix(".svg"),
        width=plot_w,
        height=plot_h,
        scale=5,
    )

In [ ]:
# Attributes to attach to Sunburst plot for each metrics
metrics_mapping = {
    "MAE": {
        "colorscale": "purples",
        "zmin": 0,
        "zmax": 50,
        "title": "Mean Absolute Error",
        "tick_suffix": "",
        "d_tick": 10,
    },
    "RMSE": {
        "colorscale": "reds",
        "zmin": 0,
        "zmax": 50,
        "title": "RMSE",
        "tick_suffix": "",
        "d_tick": 10,
    },
    "RPE": {
        "colorscale": "oranges",
        "zmin": 0,
        "zmax": 1000,
        "title": "RPE",
        "tick_suffix": " %",
        "d_tick": 100,
    },
}

#### Plot Cancer and Normal Epithelial separately

In [ ]:
all_rmse_l = []

# For each tool, calculate average absolute error across tumour purity levels
for method in tqdm(methods):
    print(f"Generating performance metrics for {method}")
    res_df = pd.read_csv(
        Path(prefix).joinpath(f"data/results/{method}.csv"), sep="\t", index_col=0
    )

    # Some values can be ~-0.00001..., replace them by 0
    res_df.clip(lower=0, inplace=True)

    # Empty list to hold metrics
    all_metrics_l = []

    for pur_lvl in tqdm(purity_levels):
        # Calculate average of the absolute difference for each purity level
        subset_truth_df = truth_df[
            truth_df["Cancer Epithelial"] == pur_lvl
        ].sort_index()
        subset_res_df = res_df[res_df.index.isin(subset_truth_df.index)].sort_index()

        pur_lvl_metrics_df = calculate_metrics(
            subset_truth_df=subset_truth_df,
            subset_res_df=subset_res_df,
            c_types=c_types,
        )
        pur_lvl_metrics_df["Purity Level"] = pur_lvl
        all_metrics_l.append(pur_lvl_metrics_df)

    # Concatenate
    all_metrics_df = pd.concat(all_metrics_l, axis=0)
    all_metrics_df = all_metrics_df.round(2)

    # Plot each metric separately
    for metric in ["RMSE"]:
        # Plot reduced tumour purity levels
        reduced_pur_lvl_df = (
            all_metrics_df.loc[
                (all_metrics_df["Purity Level"].isin(reduced_purity_levels))
                & (all_metrics_df.index == "RMSE")
            ]
            .set_index(["Purity Level"])
            .T
        )

        plot_metrics_df = all_metrics_df.loc[metric].set_index(["Purity Level"]).T

        # Plot all tumour purity levels
        all_pur_lvl_df = all_metrics_df.loc["RMSE"].set_index(["Purity Level"]).T

        # Plot heat map
        plot_rmse_heatmap(
            avg_diff_df=all_pur_lvl_df.round(1),
            outfile_name=f"figures/supp_figures/supp_fig_4c_{metric}_{method}",
            metric=metric,
            metric_suffix=metrics_mapping[metric]["tick_suffix"],
            c_types=ctypes_order,
            z_range=[metrics_mapping[metric]["zmin"], metrics_mapping[metric]["zmax"]],
            dticks=metrics_mapping[metric]["d_tick"],
            colorscale=metrics_mapping[metric]["colorscale"],
            plot_w=360,
            plot_h=125,
        )

        # Collect RMSE and method
        rmse_df = (
            all_metrics_df.loc["RMSE"].reset_index().rename(columns={"index": "metric"})
        )
        rmse_df["method"] = method
        all_rmse_l.append(rmse_df)

In [ ]:
# Save source data
all_rmse_df = pd.concat(all_rmse_l, axis=0)
all_rmse_df.replace(
    {
        "scaden": "Scaden",
        "music": "MuSiC",
        "cbx": "CBX",
        "bisque": "Bisque",
        "dwls": "DWLS",
        "epic": "EPIC",
        "cpm": "CPM",
        "bprism_v2": "BayesPrism",
    },
    inplace=True,
)
all_rmse_df["Purity Level"] = all_rmse_df["Purity Level"] * 100
all_rmse_df.to_csv(
    Path(viz_prefix).joinpath("source_data/supp_figure_4c.tsv"), sep="\t"
)

#### Collapse Cancer and Normal Epithelial into Epithelial

In [ ]:
grouped_epithelial_ctypes_order = [
    "Plasmablasts",
    "PVL",
    "CAFs",
    "Endothelial",
    "Myeloid",
    "B-cells",
    "T-cells",
    "Epithelial",
]

In [ ]:
all_rmse_l = []

# For each tool, collapse Normal Epithelial and Cancer Epithelial into Epithelial
# Then calculate average absolute error across tumour purity levels
for method in tqdm(methods):
    print(f"Generating performance metrics for {method}")
    res_df = pd.read_csv(
        Path(prefix).joinpath(f"data/results/{method}.csv"), sep="\t", index_col=0
    )

    # Some values can be ~-0.00001..., replace them by 0
    res_df[res_df < 0] = 0

    # Collapse Cancer and Epithelial in predictions and groundtruth DataFrames
    collapsed_res_df = collapse_epithelial(source_df=res_df)
    collapsed_truth_df = collapse_epithelial(source_df=truth_df)

    # Empty list to hold metrics
    all_metrics_l = []

    for pur_lvl in tqdm(purity_levels):
        # Calculate average of the absolute difference for each purity level
        subset_truth_df = collapsed_truth_df[
            truth_df["Cancer Epithelial"] == pur_lvl
        ].sort_index()
        subset_res_df = collapsed_res_df[
            collapsed_res_df.index.isin(subset_truth_df.index)
        ].sort_index()

        pur_lvl_metrics_df = calculate_metrics(
            subset_truth_df=subset_truth_df,
            subset_res_df=subset_res_df,
            c_types=grouped_epithelial_ctypes_order,
        )
        pur_lvl_metrics_df["Purity Level"] = pur_lvl
        all_metrics_l.append(pur_lvl_metrics_df)

    all_metrics_df = pd.concat(all_metrics_l, axis=0)
    all_metrics_df = all_metrics_df.round(2)

    # Plot each metric separately
    for metric in ["RMSE"]:
        # Plot reduced tumour purity levels
        reduced_pur_lvl_df = (
            all_metrics_df.loc[
                (all_metrics_df["Purity Level"].isin(reduced_purity_levels))
                & (all_metrics_df.index == "RMSE")
            ]
            .set_index(["Purity Level"])
            .T
        )

        plot_metrics_df = all_metrics_df.loc[metric].set_index(["Purity Level"]).T

        # Plot all tumour purity levels
        all_pur_lvl_df = all_metrics_df.loc["RMSE"].set_index(["Purity Level"]).T

        # Plot heatmap
        plot_rmse_heatmap(
            avg_diff_df=all_pur_lvl_df.round(1),
            outfile_name=f"figures/supp_figures/supp_fig_11a_{metric}_{method}",
            metric=metric,
            metric_suffix=metrics_mapping[metric]["tick_suffix"],
            c_types=grouped_epithelial_ctypes_order,
            z_range=[metrics_mapping[metric]["zmin"], metrics_mapping[metric]["zmax"]],
            dticks=metrics_mapping[metric]["d_tick"],
            colorscale=metrics_mapping[metric]["colorscale"],
            plot_w=310,
            plot_h=125,
        )

        # Collect RMSE and method
        rmse_df = (
            all_metrics_df.loc["RMSE"].reset_index().rename(columns={"index": "metric"})
        )
        rmse_df["method"] = method
        all_rmse_l.append(rmse_df)

In [ ]:
# Save source data
all_rmse_df = pd.concat(all_rmse_l, axis=0)
all_rmse_df.replace(
    {
        "scaden": "Scaden",
        "music": "MuSiC",
        "cbx": "CBX",
        "bisque": "Bisque",
        "dwls": "DWLS",
        "epic": "EPIC",
        "cpm": "CPM",
        "bprism_v2": "BayesPrism",
    },
    inplace=True,
)
all_rmse_df["Purity Level"] = all_rmse_df["Purity Level"] * 100
all_rmse_df.to_csv(
    Path(viz_prefix).joinpath("source_data/supp_figure_11a.tsv"), sep="\t"
)

#### For mixtures with only Cancer Epithelial

In [ ]:
no_normal_ctypes_order = [
    "Plasmablasts",
    "PVL",
    "CAFs",
    "Endothelial",
    "Myeloid",
    "B-cells",
    "T-cells",
    "Cancer Epithelial",
]

# For this plot, we'll also need predictions and groundtruth from without-Normal experiment
no_normal_prefix = "???/deconvolution_benchmarking/01_purity_levels_experiment"

no_normal_truth_df = pd.read_csv(
    Path(no_normal_prefix).joinpath("data/results/truth.csv"), sep="\t", index_col=0
)
no_normal_truth_df = no_normal_truth_df[
    [
        "B-cells",
        "CAFs",
        "Cancer Epithelial",
        "Endothelial",
        "Myeloid",
        "PVL",
        "Plasmablasts",
        "T-cells",
    ]
]

In [ ]:
all_rmse_l = []

# For each tool, collapse Normal Epithelial and Cancer Epithelial into Epithelial
# Then calculate average absolute error across tumour purity levels
for method in tqdm(methods):
    print(f"Generating performance metrics for {method}")
    res_df = pd.read_csv(
        Path(no_normal_prefix).joinpath(f"data/results/{method}.csv"),
        sep="\t",
        index_col=0,
    )

    # Some values can be ~-0.00001..., replace them by 0
    res_df.clip(lower=0)

    # Empty list to hold metrics
    all_metrics_l = []

    for pur_lvl in tqdm(purity_levels):
        # Calculate average of the absolute difference for each purity level
        subset_truth_df = no_normal_truth_df[
            no_normal_truth_df["Cancer Epithelial"] == pur_lvl
        ].sort_index()
        subset_res_df = res_df[res_df.index.isin(subset_truth_df.index)].sort_index()

        pur_lvl_metrics_df = calculate_metrics(
            subset_truth_df=subset_truth_df,
            subset_res_df=subset_res_df,
            c_types=no_normal_ctypes_order,
        )
        pur_lvl_metrics_df["Purity Level"] = pur_lvl
        all_metrics_l.append(pur_lvl_metrics_df)

    all_metrics_df = pd.concat(all_metrics_l, axis=0)
    all_metrics_df = all_metrics_df.round(2)

    # Plot each metric separately
    for metric in ["RMSE"]:
        # Plot reduced tumour purity levels
        reduced_pur_lvl_df = (
            all_metrics_df.loc[
                (all_metrics_df["Purity Level"].isin(reduced_purity_levels))
                & (all_metrics_df.index == "RMSE")
            ]
            .set_index(["Purity Level"])
            .T
        )

        plot_metrics_df = all_metrics_df.loc[metric].set_index(["Purity Level"]).T

        # Plot all tumour purity levels
        all_pur_lvl_df = all_metrics_df.loc["RMSE"].set_index(["Purity Level"]).T

        # Plot heatmap
        plot_rmse_heatmap(
            avg_diff_df=all_pur_lvl_df,
            outfile_name=f"figures/supp_figures/supp_fig_11b_{metric}_all_{method}",
            metric=metric,
            metric_suffix=metrics_mapping[metric]["tick_suffix"],
            c_types=no_normal_ctypes_order,
            z_range=[metrics_mapping[metric]["zmin"], metrics_mapping[metric]["zmax"]],
            dticks=metrics_mapping[metric]["d_tick"],
            colorscale=metrics_mapping[metric]["colorscale"],
            plot_w=310,
            plot_h=125,
        )

        # Collect RMSE and method
        rmse_df = (
            all_metrics_df.loc["RMSE"].reset_index().rename(columns={"index": "metric"})
        )
        rmse_df["method"] = method
        all_rmse_l.append(rmse_df)

In [ ]:
# Save source data
all_rmse_df = pd.concat(all_rmse_l, axis=0)
all_rmse_df.replace(
    {
        "scaden": "Scaden",
        "music": "MuSiC",
        "cbx": "CBX",
        "bisque": "Bisque",
        "dwls": "DWLS",
        "epic": "EPIC",
        "cpm": "CPM",
        "bprism_v2": "BayesPrism",
    },
    inplace=True,
)
all_rmse_df.to_csv(
    Path(viz_prefix).joinpath("source_data/supp_figure_11b.tsv"), sep="\t"
)

## [Extended Data Fig 1a]. Bray Curtis dissimilarity across tumour purity levels

In [ ]:
methods = [
    "bprism_v2",
    "scaden",
    "music",
    "cpm",
    "cbx",
    "hspe",
    "epic",
    "bisque",
    "dwls",
]

# We don't need to show all 19 tumour purities for each tool. Just half of it will be fine
# Getting tumour purity levels with intervals of 15% instead of 10%
reduced_purity_levels = np.arange(0.05, 1, 0.15).round(3).tolist()

#### Calculate Bray-Curtis dissmilarity index across tumour purity levels

In [ ]:
bray_curtis_l = []

for method in tqdm(methods):
    res_df = pd.read_csv(
        Path(prefix).joinpath(f"data/results/{method}.csv"), sep="\t", index_col=0
    )
    res_df = res_df[c_types]

    # Clip tiny negative numbers to 0
    res_df.clip(lower=0, inplace=True)

    # Check if indexes match
    assert (res_df.sort_index().index == truth_df.sort_index().index).all()
    assert (res_df.sort_index().columns == truth_df.sort_index().columns).all()

    # Iterate over res_df and calculate Bray-Curtis index
    for sample_id in res_df.index:
        bray_curtis_dissi = braycurtis(res_df.loc[sample_id], truth_df.loc[sample_id])
        bray_curtis_l.append(
            (
                sample_id,
                bray_curtis_dissi,
                truth_df.loc[sample_id, "Cancer Epithelial"],
                method,
            )
        )

# Concatenate all rmse dataframes
bray_curtis_df = pd.DataFrame(
    bray_curtis_l, columns=["Mixture ID", "Bray Curtis Dissi", "Purity Level", "Method"]
).set_index(["Mixture ID"])

# Rename method names
bray_curtis_df.replace(
    {
        "scaden": "Scaden",
        "music": "MuSiC",
        "cbx": "CBX",
        "bisque": "Bisque",
        "dwls": "DWLS",
        "epic": "EPIC",
        "cpm": "CPM",
        "bprism_v2": "BayesPrism",
    },
    inplace=True,
)

# Convert purity level to % scale and categorical
bray_curtis_df["Purity Level"] = (bray_curtis_df["Purity Level"] * 100).astype(int)
bray_curtis_df["Purity Level"] = bray_curtis_df["Purity Level"].astype("category")
bray_curtis_df["Purity Level"].cat.reorder_categories(
    [int(i * 100) for i in purity_levels], inplace=True
)

#### Heatmaps of median Bray Curtis dissimilarity across tumour purity levels

In [ ]:
# Get median of bray-curtis at each tumour purity level
median_bray_curtis_df = bray_curtis_df.groupby(["Method", "Purity Level"]).agg(
    ["median"]
)

# 1st level of multi-index column is redundant, drop it and reset index
median_bray_curtis_df = median_bray_curtis_df.droplevel(0, axis=1).reset_index()

# Pivot wider so we have purity levels as columns (x-axis) and methods as rows (y-axis)
pivot_median_bray_curtis_df = median_bray_curtis_df.pivot(
    index="Method", columns="Purity Level", values="median"
)

# y-axis order is reverse
pivot_median_bray_curtis_df = pivot_median_bray_curtis_df.loc[methods_order[::-1], :]

In [ ]:
# Save source data
median_bray_curtis_df.to_csv(
    Path(viz_prefix).joinpath("source_data/supp_figure_4a.tsv"), sep="\t"
)

In [ ]:
fig = ff.create_annotated_heatmap(
    z=pivot_median_bray_curtis_df.values,
    annotation_text=pivot_median_bray_curtis_df.values.round(2),
    zmin=0,
    zmax=1,
    x=pivot_median_bray_curtis_df.columns.tolist(),
    y=pivot_median_bray_curtis_df.index.tolist(),
    colorscale="teal",
    showscale=True,
    hoverinfo="text",
    text=pivot_median_bray_curtis_df.round(2).values,
    colorbar=dict(
        title="Bray-Curis<br>Dissimilarity",
        titlefont_size=14,
        ticks="outside",
        ticksuffix="",
        tickfont_size=12,
        dtick=0.1,
        orientation="v",
    ),
)

fig["layout"].update(
    margin=dict(t=0, l=0, r=0, b=0),
    plot_bgcolor="rgba(0,0,0,0)",
    font=dict(size=12, color="black"),
    xaxis=dict(
        title="Tumour purity levels (%)",
        titlefont_size=14,
        title_standoff=5,
        ticks="outside",
        showticklabels=True,
        tickmode="array",
        tickwidth=0.75,
        ticklen=3,
        tickvals=[int(i * 100) for i in purity_levels],
        tickfont_size=12,
        linecolor="black",
        linewidth=0.75,
        side="bottom",
    ),
    yaxis=dict(
        title="Methods",
        titlefont_size=14,
        title_standoff=1,
        linecolor="black",
        linewidth=0.75,
        ticks="outside",
        tickwidth=0.75,
        ticklen=3,
        tickfont_size=12,
        categoryorder="array",
        categoryarray=methods_order,
    ),
)

fig.write_image(
    Path("figures/supp_figures/supp_fig_4a").with_suffix(".svg"),
    width=900,
    height=300,
    scale=5,
)